In [3]:
#Functions for parsing columns of the spreadsheet on Santa Clara County School Meals
def fcnparse_opendays(stringopendays):
    print(stringopendays)
    
def fcnparse_siteaddress(full_address):
    
    tmp = full_address.split(', ')
    address = full_address#tmp[0]
    city = ""#tmp[1]
    tmp2 = ""#tmp[2].split(' ')
    state = ""#tmp2[0]
    zipcode = ""#tmp2[1]    
    return address, city, state, zipcode

def fcnparse_opendays(attributesOPENDAYS):
    return ["Monday","Tuesday","Thursday"]

def fcnparse_opentimes(attributesOPENTIMES):
    return "2:00 PM", "6:00 PM"

def fcnparse_mealtypes(attributesMEALTYPES):
    return ["In person","Drive Thru"]

In [4]:
#


post_url_services = 'https://api.airtable.com/v0/applQOkth8R2ns3qo/services'
post_url_organizations = 'https://api.airtable.com/v0/applQOkth8R2ns3qo/organizations'
post_url_address = 'https://api.airtable.com/v0/applQOkth8R2ns3qo/address'
post_url_schedule = 'https://api.airtable.com/v0/applQOkth8R2ns3qo/schedule'

post_headers = {
    'Authorization' : 'Bearer keyFaCMgt8zX71GnJ',
    'Content-Type': 'application/json'
}

f = open('/Users/Ashwin/Documents/BAC/20200919cfsj/Santa Clara Scrape 10_1 - Food School Meal Sites .csv')
csv_f = csv.reader(f)

#Skip header row
count = 1
for row in csv_f:
    if count == 1:
        break

count = 1
for row in csv_f:
    #print(row)
    rowcount = 0
    for elem in row:
        print(rowcount, elem)
        rowcount = rowcount+1
    
    #Parse data for ORGANIZATIONS sheet: attributes.SITESCHOOLDISTRICT
    attributesSITESCHOOLDISTRICT = row[4]
    airtableORGANIZATIONS = {"fields": {"name": attributesSITESCHOOLDISTRICT}}

    #Parse data for SERVICES sheet: attributes.SITENAME, attributes.MEALTYPES
    attributesSITENAME = row[5]
    attributesMEALTYPES = fcnparse_mealtypes(row[10])
    airtableSERVICES = {"fields": {"Name": attributesSITENAME, "service_options": attributesMEALTYPES}}
    
    #Parse data for ADDRESS sheet: attributes.SITEADDRESS   
    attributesSITEADDRESS = row[6]
    address, city, state, zipcode = fcnparse_siteaddress(attributesSITEADDRESS)
    print(address, city, state, zipcode)
    airtableADDRESS = {"fields": {"address_1": address,"city": city,"State": state,"Zip Code": zipcode}} 
        
    #Parse data for SCHEDULE sheet: attributes.OPENDAYS, attributes.OPENTIMES
    attributesOPENDAYS = row[7]
    attributesOPENTIMES = row[8]
    opendays = fcnparse_opendays(attributesOPENDAYS)
    opensat, closesat = fcnparse_opentimes(attributesOPENTIMES)
    airtableSCHEDULE = {"fields": {"weekday": opendays, "opens_at": opensat, "closes_at": closesat}} 
    
    #Upload to airtable
    print((requests.post(post_url_organizations, headers = post_headers, json = airtableORGANIZATIONS)).status_code)    
    print((requests.post(post_url_services, headers = post_headers, json = airtableSERVICES)).status_code)    
    print((requests.post(post_url_address, headers = post_headers, json = airtableADDRESS)).status_code)    
    print((requests.post(post_url_schedule, headers = post_headers, json = airtableSCHEDULE)).status_code)
     
 
    
    if count == 1:
        break    

0 0
1 1
2 1
3 1
4 Alum Rock Union SD
5 Painter/Sheppard MS
6 480 Rough & Ready Rd. San Jose 
 
 

7 Monday through Thursday


De lunes a jueves
8 11:00 a.m. to 1:00 p.m.
9 On-going through school June 9th
10 Lunch and Breakfast
11 School
12 Open
13 The school meal distribution program provides food for low-income families whose children depend on the lunch or breakfast they get from the cafeteria.
14 
15 
16 https://resources.finalsite.net/images/v1584407264/arusdorg/ctuat3itibpp3impkobc/FamilyLetter3-6-20.pdf
17 San Jose
18 5
19 1588782424000
20 FME
21 *updated 5.4.20 at 4:12 p.m.
22 EOC-FCMS-0000000001
23 Curbside pickup or walk through.
24 -13563497.92
25 4491488.531
480 Rough & Ready Rd. San Jose 
 
 
   
200
200
200
200


In [35]:
from OSMPythonTools.nominatim import Nominatim
nominatim = Nominatim()
#myoutput = nominatim.query('1680 Foley ave')
#print(myoutput.toJSON())

myoutput = nominatim.query('1680 Foley Ave, California')
myjson = myoutput.toJSON()[0]
print(myjson)
print(type(myjson))
full_address = myjson['display_name']
print(full_address)
print(type(full_address))

tmp = full_address.split(', ')
print(tmp)
address = tmp[1]+" "+tmp[2]
city = tmp[-5:-4][0]
state = tmp[-3:-2][0]
zipcode = tmp[-2:-1][0]

print(address)
print(city)
print(state)
print(zipcode)

[nominatim] downloading data: 1680 Foley Ave, California
{'place_id': 98165860, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 38689781, 'boundingbox': ['37.329458', '37.3321184', '-121.840532', '-121.837463'], 'lat': '37.3308121', 'lon': '-121.83900825492344', 'display_name': 'O.S. Hubbard Elementary School, 1680, Foley Avenue, Little Saigon, Little Portugal, San Jose, Santa Clara County, California, 95122, United States of America', 'class': 'amenity', 'type': 'school', 'importance': 0.42099999999999904, 'icon': 'https://nominatim.openstreetmap.org/images/mapicons/education_school.p.20.png'}
<class 'dict'>
O.S. Hubbard Elementary School, 1680, Foley Avenue, Little Saigon, Little Portugal, San Jose, Santa Clara County, California, 95122, United States of America
<class 'str'>
['O.S. Hubbard Elementary School', '1680', 'Foley Avenue', 'Little Saigon', 'Little Portugal', 'San Jose', 'Santa Clara County', 'California', '9